In [1]:
import threading  # threading modülünü içe aktarır
import time        # Zamanlama için time modülü

def say_hello(name, delay):
    """Belirli aralıklarla selam mesajı yazdırır"""
    for i in range(3):  # 3 kez döngü
        time.sleep(delay)  # Belirtilen saniye kadar bekler
        print(f"Merhaba {name}! (İşlem: {threading.current_thread().name})")  # Hangi thread'in çalıştığını gösterir

# Thread'leri oluşturma
thread1 = threading.Thread(
    target=say_hello,  # Çalıştırılacak fonksiyon
    args=("Ali", 1),   # Fonksiyona gönderilecek argümanlar
    name="Thread-1"    # Thread ismi (opsiyonel)
)
thread2 = threading.Thread(
    target=say_hello,
    args=("Ayşe", 0.5),
    name="Thread-2"
)

# Thread'leri başlatma
thread1.start()  # Thread'i çalıştırır (fonksiyon paralel olarak başlar)
thread2.start()

# Ana thread'in diğer thread'leri beklemesi
thread1.join()  # Thread1'in bitmesini bekler
thread2.join()  # Thread2'in bitmesini bekler

print("Tüm thread'ler tamamlandı!")  # Tüm thread'ler bittikten sonra çalışır


Merhaba Ayşe! (İşlem: Thread-2)
Merhaba Ali! (İşlem: Thread-1)
Merhaba Ayşe! (İşlem: Thread-2)
Merhaba Ayşe! (İşlem: Thread-2)
Merhaba Ali! (İşlem: Thread-1)
Merhaba Ali! (İşlem: Thread-1)
Tüm thread'ler tamamlandı!


In [2]:
import threading

balance = 1000  # Paylaşılan değişken
lock = threading.Lock()  # Kilitleme nesnesi oluşturulur

def withdraw(amount):
    """Bakiyeden para çekme işlemi"""
    global balance  # Global değişkene erişim
    with lock:  # Kritik bölgeyi kilitle
        if balance >= amount:
            new_balance = balance - amount  # Geçici hesaplama
            time.sleep(0.1)  # Sanal işlem gecikmesi
            balance = new_balance  # Güncellenmiş bakiye
            print(f"Çekildi: {amount}, Kalan: {balance}")
        else:
            print("Yetersiz bakiye!")

# Thread'ler oluşturma
threads = []
for _ in range(5):
    t = threading.Thread(target=withdraw, args=(300,))
    threads.append(t)

# Tüm thread'leri başlatma
for t in threads:
    t.start()

# Tüm thread'leri bekletme
for t in threads:
    t.join()

print(f"Son bakiye: {balance}")


Çekildi: 300, Kalan: 700
Çekildi: 300, Kalan: 400
Çekildi: 300, Kalan: 100
Yetersiz bakiye!
Yetersiz bakiye!
Son bakiye: 100


In [2]:
import queue
import threading
import time

def producer(q, items):
    """Üretici: Kuyruğa veri ekler"""
    for i in range(items):
        time.sleep(0.5)
        item = f"Ürün-{i}"
        q.put(item)  # Kuyruğa eleman ekler
        print(f"Üretildi: {item}")

def consumer(q):
    """Tüketici: Kuyruktan veri alır"""
    while True:
        item = q.get()  # Kuyruktan eleman alır (boşsa bekler)
        if item is None:  # Durma sinyali
            break
        print(f"Tüketildi: {item}")
        time.sleep(1)
        q.task_done()  # Görev tamamlandı sinyali

# Güvenli kuyruk oluşturma
q = queue.Queue()

# Thread'leri başlatma
producer_thread = threading.Thread(target=producer, args=(q, 5))
consumer_thread = threading.Thread(target=consumer, args=(q,))

producer_thread.start()
consumer_thread.start()

# Üretici bitince tüketiciyi durdurma
producer_thread.join()
q.put(None)  # Tüketiciye dur sinyali

consumer_thread.join()
print("İşlem tamamlandı!")


Üretildi: Ürün-0
Tüketildi: Ürün-0


Üretildi: Ürün-1
Üretildi: Ürün-2Tüketildi: Ürün-1

Üretildi: Ürün-3
Tüketildi: Ürün-2Üretildi: Ürün-4

Tüketildi: Ürün-3
Tüketildi: Ürün-4
İşlem tamamlandı!


In [3]:
import threading
import requests
import time
from urllib.parse import urlparse

# İndirilecek dosya listesi
urls = [
    "https://example.com/file1.zip",
    "https://example.com/file2.zip",
    "https://example.com/file3.zip"
]

def download_file(url):
    """Dosyayı indirip yerel klasöre kaydeder"""
    filename = urlparse(url).path.split("/")[-1]  # URL'den dosya adını alır
    print(f"İndirme başlıyor: {filename}")

    try:
        response = requests.get(url, stream=True)  # Akış modunda istek yapar
        with open(filename, "wb") as f:  # Dosyayı yazma modunda açar
            for chunk in response.iter_content(8192):  # 8KB'lık parçalar halinde
                f.write(chunk)  # Dosyaya yazar
        print(f"Tamamlandı: {filename}")
    except Exception as e:
        print(f"Hata {filename}: {str(e)}")

# Thread pool oluşturma
threads = []
start_time = time.time()

for url in urls:
    t = threading.Thread(target=download_file, args=(url,))
    threads.append(t)
    t.start()

# Tüm thread'leri bekletme
for t in threads:
    t.join()

print(f"Toplam süre: {time.time() - start_time:.2f} saniye")


İndirme başlıyor: file1.zip
İndirme başlıyor: file2.zip
İndirme başlıyor: file3.zip
Tamamlandı: file3.zip
Tamamlandı: file1.zip
Tamamlandı: file2.zip
Toplam süre: 0.80 saniye


In [4]:
import threading

lock1 = threading.Lock()
lock2 = threading.Lock()

def thread1():
    with lock1:
        time.sleep(0.1)
        with lock2:  # lock2'yi beklerken lock1'i elinde tutar
            print("Thread 1")

def thread2():
    with lock2:
        time.sleep(0.1)
        with lock1:  # lock1'i beklerken lock2'yi elinde tutar
            print("Thread 2")

# Bu durumda deadlock oluşur!


In [5]:
import threading
import time

lock1 = threading.Lock()
lock2 = threading.Lock()

def thread1():
    print("Thread 1, lock1'i almaya çalışıyor.")
    if lock1.acquire(timeout=2):
        print("Thread 1, lock1'i aldı.")
        time.sleep(0.1)
        
        print("Thread 1, lock2'yi almaya çalışıyor.")
        if lock2.acquire(timeout=2):
            print("Thread 1, lock2'yi aldı.")
            print("Thread 1 çalışıyor.")
            lock2.release()
            print("Thread 1, lock2'yi serbest bıraktı.")
        else:
            print("Thread 1, lock2'yi 2 saniye içinde alamadı. Kilidi serbest bırakıyor.")
            
        lock1.release()
        print("Thread 1, lock1'i serbest bıraktı.")
    else:
        print("Thread 1, lock1'i 2 saniye içinde alamadı.")

def thread2():
    print("Thread 2, lock2'yi almaya çalışıyor.")
    if lock2.acquire(timeout=2):
        print("Thread 2, lock2'yi aldı.")
        time.sleep(0.1)

        print("Thread 2, lock1'i almaya çalışıyor.")
        if lock1.acquire(timeout=2):
            print("Thread 2, lock1'i aldı.")
            print("Thread 2 çalışıyor.")
            lock1.release()
            print("Thread 2, lock1'i serbest bıraktı.")
        else:
            print("Thread 2, lock1'i 2 saniye içinde alamadı. Kilidi serbest bırakıyor.")

        lock2.release()
        print("Thread 2, lock2'yi serbest bıraktı.")
    else:
        print("Thread 2, lock2'yi 2 saniye içinde alamadı.")

# Deadlock'ı tetiklemek için her iki thread'i de başlatın
t1 = threading.Thread(target=thread1)
t2 = threading.Thread(target=thread2)

t1.start()
t2.start()

t1.join()
t2.join()

print("Program tamamlandı.")

Thread 1, lock1'i almaya çalışıyor.
Thread 1, lock1'i aldı.
Thread 2, lock2'yi almaya çalışıyor.
Thread 2, lock2'yi aldı.
Thread 1, lock2'yi almaya çalışıyor.
Thread 2, lock1'i almaya çalışıyor.


Thread 1, lock2'yi 2 saniye içinde alamadı. Kilidi serbest bırakıyor.
Thread 1, lock1'i serbest bıraktı.
Thread 2, lock1'i aldı.
Thread 2 çalışıyor.
Thread 2, lock1'i serbest bıraktı.
Thread 2, lock2'yi serbest bıraktı.
Program tamamlandı.


In [8]:
import multiprocessing
import threading
import time
import logging

# Loglama ayarlarını yapılandır
logging.basicConfig(
    level=logging.INFO, # Bilgilendirme seviyesindeki logları gösterir
    format='%(asctime)s - %(threadName)s - %(processName)s - %(message)s' # Log formatını belirler
)

# --- CPU yoğun işlem (Matematiksel hesaplama) ---
def calculate_prime_numbers(n):
    """Belirli bir sayıya kadar olan asal sayıları bulan CPU yoğun fonksiyon."""
    primes = []
    for num in range(2, n + 1): # 2'den n'ye kadar döngü
        is_prime = True
        for i in range(2, int(num**0.5) + 1): # Sayının kareköküne kadar bölen arar
            if num % i == 0:
                is_prime = False
                break
        if is_prime:
            primes.append(num)
    
    logging.info(f"Hesaplama tamamlandı. Toplam {len(primes)} adet asal sayı bulundu.")
    return primes

def process_worker():
    """Multiprocessing için kullanılacak işçi fonksiyonu."""
    logging.info("Process başladı.") # Sürecin başladığını logla
    calculate_prime_numbers(300000) # CPU yoğun işlemi çağır
    logging.info("Process bitti.") # Sürecin bittiğini logla

# --- Threading ile eş zamanlı işlem (I/O yoğun) ---
def io_task():
    """I/O yoğun bir işlemi simüle eden fonksiyon."""
    thread_name = threading.current_thread().name # Mevcut thread'in adını al
    logging.info(f"'{thread_name}' başladı.")
    time.sleep(2) # 2 saniye bekleyerek I/O işlemi simüle et
    logging.info(f"'{thread_name}' bitti.")

# Ana fonksiyon
def main():
    logging.info("Ana program başladı.") # Programın başladığını logla
    
    # Aktif thread'leri kontrol et
    active_threads = threading.enumerate()
    logging.info(f"Başlangıçta aktif olan thread'ler: {[t.name for t in active_threads]}")
    
    # Multiprocessing ile CPU yoğun işlemi çalıştır
    logging.info("Multiprocessing ile CPU yoğun işlem başlatılıyor.")
    p = multiprocessing.Process(target=process_worker, name="CPU-Process")
    p.start() # Süreci başlat
    
    # Threading ile I/O yoğun işlemi çalıştır
    logging.info("Threading ile I/O yoğun işlem başlatılıyor.")
    t = threading.Thread(target=io_task, name="IO-Thread")
    t.start() # Thread'i başlat
    
    # Aktif thread'leri kontrol et (yeni thread eklendi)
    active_threads_after_start = threading.enumerate()
    logging.info(f"Yeni bir thread başlatıldıktan sonra aktif thread'ler: {[t.name for t in active_threads_after_start]}")

    # İşlemlerin bitmesini bekle
    p.join()
    t.join()
    
    logging.info("Ana program bitti.")

if __name__ == "__main__":
    main()

2025-08-14 09:30:37,140 - MainThread - MainProcess - Ana program başladı.
2025-08-14 09:30:37,141 - MainThread - MainProcess - Başlangıçta aktif olan thread'ler: ['MainThread', 'IOPub', 'Heartbeat', 'Thread-1 (_watch_pipe_fd)', 'Thread-2 (_watch_pipe_fd)', 'Control', 'IPythonHistorySavingThread']
2025-08-14 09:30:37,141 - MainThread - MainProcess - Multiprocessing ile CPU yoğun işlem başlatılıyor.
2025-08-14 09:30:37,145 - MainThread - MainProcess - Threading ile I/O yoğun işlem başlatılıyor.
2025-08-14 09:30:37,148 - IO-Thread - MainProcess - 'IO-Thread' başladı.
2025-08-14 09:30:37,146 - MainThread - CPU-Process - Process başladı.
2025-08-14 09:30:37,148 - MainThread - MainProcess - Yeni bir thread başlatıldıktan sonra aktif thread'ler: ['MainThread', 'IOPub', 'Heartbeat', 'Thread-1 (_watch_pipe_fd)', 'Thread-2 (_watch_pipe_fd)', 'Control', 'IPythonHistorySavingThread', 'IO-Thread']
2025-08-14 09:30:37,725 - MainThread - CPU-Process - Hesaplama tamamlandı. Toplam 25997 adet asal sayı